In [ ]:
%pip install --upgrade pandas
%pip install --upgrade numpy
%pip install --upgrade matplotlib
%pip install --upgrade seaborn
%pip install --upgrade scikit-learn-intelex
%pip install --upgrade dpcpp-cpp-rt
%pip install --upgrade xgboost
%pip install --upgrade sklearn-ex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

# Importing Libraries

In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('all.csv')
dummies=pd.get_dummies(df[["name","datetime","preciptype","conditions","description","icon","stations"]])
tranformed_x=dummies
tranformed_x


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['names'].unique()

In [ ]:
df.dtypes

In [ ]:
df['names'].value_counts()

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(df.corr(),annot=True)

# defining x and y axis 

In [9]:
x=df.copy(["preciptype","temp","humidity","conditions","rain"],axis=1)
y= df["preciptype"]

KeyError: "['rain'] not found in axis"

In [ ]:
acc = []
model = []

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(tranformed_x,y,test_size = 0.2)

# Ensemble learning

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

tree_model = DecisionTreeRegressor()
rf_model = RandomForestRegressor()

tree_model.fit(Xtrain, Ytrain)
rf_model.fit(Xtrain, Ytrain)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearnex import patch_sklearn
patch_sklearn()

LogReg = LogisticRegression(random_state=2)

LogReg.fit(Xtrain,Ytrain)

predicted_values = LogReg.predict(Xtest)

x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('Logistic Regression')
print("Logistic Regression's Accuracy is: ", x)

print(classification_report(Ytest,predicted_values))

# random forest

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=20, random_state=0)
RF.fit(Xtrain,Ytrain)

predicted_values = RF.predict(Xtest)

x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('RF')
print("RF's Accuracy is: ", x)

print(classification_report(Ytest,predicted_values))

# XG BOOST

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
import xgboost as xgb
XB = xgb.XGBClassifier()
xgb_model=XB.fit(Xtrain,Ytrain)

predicted_values = XB.predict(Xtest)

x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('XGBoost')
print("XGBoost's Accuracy is: ", x)

print(classification_report(Ytest,predicted_values))